In [45]:
# 環境ファイルからAPI KEYを指定する
import os
from dotenv import load_dotenv

# .envファイルのパスを指定して読み込む
load_dotenv('.env')

# 環境変数を利用する
NEWS_API_KEY = os.getenv('NEWS_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [46]:
# 日時の取得と計算

from datetime import datetime, timedelta# 現在時刻などの時間を扱う機能をインポート

date_to = datetime.now().date()
date_from_param = date_to - timedelta(days=5)

print(date_to)
print(date_from_param)

2023-11-12
2023-11-07


In [47]:
#News APIから記事情報を取得
from newsapi import NewsApiClient
import pandas as pd

# Init
newsapi = NewsApiClient(api_key= NEWS_API_KEY)

all_articles = newsapi.get_everything(q='AI',
                                      #sources='bbc-news',
                                      domains='bbc.co.uk,bbc.com',
                                      from_param=date_from_param,
                                      to=date_to,
                                      language='en',
                                      #sort_by='relevancy',
                                      #page=2
                                      )

print(all_articles['articles'])

[{'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': 'https://www.facebook.com/bbcnews', 'title': 'AI: Fears hundreds of children globally used in naked images', 'description': 'Indecent artificial intelligence-generated pictures are believed to have swept across the globe.', 'url': 'https://www.bbc.co.uk/news/uk-wales-67344916', 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/BB50/production/_131225974_miriamaladib1.jpg', 'publishedAt': '2023-11-07T22:53:37Z', 'content': "The mother of a girl whose photo was used in AI-generated naked images says hundreds of parents have told her their children are also victims.\r\nMiriam Al Adib's daughter was one of several children f… [+3788 chars]"}, {'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': 'https://www.facebook.com/bbcnews', 'title': 'Actors union Sag strikes tentative deal to end strike', 'description': 'Sag-Aftra has been calling for better pay for actors and safeguards on the use of AI.', 'url': 'htt

In [23]:
# データフレーム作成
data_articles = pd.DataFrame(columns=['日時', 'タイトル', 'コンテンツ', 'URL'])
for i in range(len(all_articles['articles'])):
    _d = pd.DataFrame()
    _d['日時'] = [all_articles['articles'][i]['publishedAt'][:10]]
    _d['タイトル'] = [all_articles['articles'][i]['title']]
    _d['コンテンツ'] = [all_articles['articles'][i]['description']]
    _d['URL'] = [all_articles['articles'][i]['url']]
    data_articles = pd.concat([data_articles, _d], ignore_index=True)

data_articles

,日時,タイトル,コンテンツ,URL
0,2023-11-07,AI: Fears hundreds of children globally used i...,Indecent artificial intelligence-generated pic...,https://www.bbc.co.uk/news/uk-wales-67344916
1,2023-11-09,Actors union Sag strikes tentative deal to end...,Sag-Aftra has been calling for better pay for ...,https://www.bbc.co.uk/news/entertainment-arts-...
2,2023-11-07,Can AI cut humans out of contract negotiations?,Tech firms are testing artificial intelligence...,https://www.bbc.co.uk/news/business-67238386
3,2023-11-08,Can you tell the real pictures from those made...,Do you think you have what it takes to spot th...,https://www.bbc.co.uk/bitesize/articles/zqdxywx
4,2023-11-09,Apple co-founder Steve Wozniak in hospital - r...,The 73-year-old passed out at a business confe...,https://www.bbc.co.uk/news/technology-67366306
5,2023-11-09,The race to buy AI website addresses,"As more firms use artificial intelligence, the...",https://www.bbc.co.uk/news/business-67284936
6,2023-11-07,Hollywood strike: Actors union Sag says issues...,"Now in its 117th day, US actors' union says th...",https://www.bbc.co.uk/news/entertainment-arts-...
7,2023-11-08,Bad Bunny not happy about AI track using his v...,"The track, called NostalgIA, has gone viral on...",https://www.bbc.co.uk/news/newsbeat-67355245
8,2023-11-07,Rooney speaks of drink struggles in Burrow pod...,Ex-England captain Rooney opens up to Rob Burr...,https://www.bbc.co.uk/news/uk-england-leeds-67...
9,2023-11-11,Actors' deal includes pay rises and intimacy c...,The US actors' union says its national board h...,https://www.bbc.co.uk/news/world-us-canada-673...


In [49]:
# タイトルの和訳
#ChatGPTでの翻訳機能
import openai

openai.api_key = OPENAI_API_KEY

def transrate_title_to_japanese(content_text_to_gpt):
    request_to_gpt = "以下の英語の記事のタイトルを日本語で翻訳してください" + content_text_to_gpt
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": request_to_gpt},
        ],
    )


    output_content = response.choices[0]["message"]["content"].strip()
    return output_content

In [53]:
print(transrate_title_to_japanese(data_articles[1])

SyntaxError: incomplete input (1434247845.py, line 1)

In [48]:
# タイトルの和訳
#ChatGPTでの翻訳機能
import openai

openai.api_key = OPENAI_API_KEY

def transrate_title_to_japanese(content_text_to_gpt):
    request_to_gpt = "以下の英語の記事のタイトルを日本語で翻訳してください" + content_text_to_gpt
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": request_to_gpt},
        ],
    )


    output_content = response.choices[0]["message"]["content"].strip()
    return output_content


# "日本語タイトル"のコラムを新規に作成して追加
def add_japanese_column(data_articles):

    _list = []
    for i in range(len(data_articles)):
        # タイトルを日本語に翻訳
        output_japanese_title = transrate_title_to_japanese(data_articles["タイトル"][i])
        # 翻訳されたタイトルをリストに追加
        _list.append(output_japanese_title)

    # "日本語タイトル"を新規に作成してリストを入れる
    data_articles.loc[:, "日本語タイトル"] = _list  # .locを使って、明示的にdata_articlesを直接変更したつもりだが、 SettingWithCopyWarningが消えない
    
    return data_articles


# お試しで始めの5行だけ実施
test_data_articles = data_articles.head(5)

add_japanese_column(test_data_articles)


C:\Users\user\AppData\Local\Temp\ipykernel_3204\3827705359.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_articles.loc[:, "日本語タイトル"] = _list  # .locを使って、明示的にdata_articlesを直接変更したつもりだが、 SettingWithCopyWarningが消えない


,日時,タイトル,コンテンツ,URL,日本語タイトル
0,2023-11-07,AI: Fears hundreds of children globally used i...,Indecent artificial intelligence-generated pic...,https://www.bbc.co.uk/news/uk-wales-67344916,AI: 全世界で何百人もの子供が裸の画像に使用されるとの恐れ
1,2023-11-09,Actors union Sag strikes tentative deal to end...,Sag-Aftra has been calling for better pay for ...,https://www.bbc.co.uk/news/entertainment-arts-...,俳優組合SAGがストライキを終結させるための暫定的な合意に達する
2,2023-11-07,Can AI cut humans out of contract negotiations?,Tech firms are testing artificial intelligence...,https://www.bbc.co.uk/news/business-67238386,AIが契約交渉から人間を排除することはできるか？
3,2023-11-08,Can you tell the real pictures from those made...,Do you think you have what it takes to spot th...,https://www.bbc.co.uk/bitesize/articles/zqdxywx,AIによって生成された画像と本物の画像を見分けることができますか？
4,2023-11-09,Apple co-founder Steve Wozniak in hospital - r...,The 73-year-old passed out at a business confe...,https://www.bbc.co.uk/news/technology-67366306,アップル共同創業者のスティーブ・ウォズニアックが入院中 - 報道


In [38]:
test_data_articles[["日時","日本語タイトル"]]

,日時,日本語タイトル
0,2023-11-07,AI: 数百人の子供が裸の画像に使用されているとの恐れ
1,2023-11-09,俳優組合のSAG、ストライキを終えるための暫定合意に到達
2,2023-11-07,AIが契約交渉から人間を排除できるでしょうか？
3,2023-11-08,AIによって作成された画像と本物の画像を見分けることができますか？
4,2023-11-09,アップルの共同創設者、スティーブ・ウォズニアック氏が入院中 - 報道


In [39]:
# 指定した日本語タイトルを選択する部分を作りたい

for i in test_data_articles.iterrows():
    print(i[0], i[1]["日時"], i[1]["日本語タイトル"])

# indexと日本語タイトルをセットにしておけば、indexと使ってURLを抽出できるはず

0 2023-11-07 AI: 数百人の子供が裸の画像に使用されているとの恐れ
1 2023-11-09 俳優組合のSAG、ストライキを終えるための暫定合意に到達
2 2023-11-07 AIが契約交渉から人間を排除できるでしょうか？
3 2023-11-08 AIによって作成された画像と本物の画像を見分けることができますか？
4 2023-11-09 アップルの共同創設者、スティーブ・ウォズニアック氏が入院中 - 報道


In [54]:
#BeautifulSoupのインストール
import requests
from bs4 import BeautifulSoup
REQUEST_URL = 'https://www.bbc.com/news/technology-67285315' #アクセス先をREQUEST_URLを代入します。
res = requests.get(REQUEST_URL) #リクエストしたデータをresに代入します。

#BBCでのスクレイピング機能
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text,"html.parser")
content = soup.select('#main-content > article')
lines = []
for t in content:
    lines.append(t.text)
print(lines)
article = ' '.join(lines) #辞書型からテキストに変更
print(article)

['Rishi Sunak: AI firms cannot \'mark their own homework\'Published1 NovemberShareclose panelShare pageCopy linkAbout sharingThis video can not be playedTo play this video you need to enable JavaScript in your browser.Media caption, Rishi Sunak says Elon Musk \'can be valuable\' in the AI conversationBy Tom Gerken & Imran Rahman-JonesTechnology reportersMonitoring the risks posed by artificial intelligence (AI) is too important to be left to big tech firms, Prime Minister Rishi Sunak has said. He told the BBC that governments needed to take action and AI firms could not be left to "mark their own homework".He was speaking ahead of the AI Safety Summit, where a global declaration on managing AI risks has been announced.King Charles told delegates the issue required "urgency, unity and collective strength".It comes amid growing concerns about highly advanced forms of AI with as-yet unknown capabilities. So far countries are only starting to address the potential risks, which may include 

In [59]:
#ChatGPTでの翻訳機能
import openai

openai.api_key = OPENAI_API_KEY

def run_gpt(content_to_text):
    request_to_gpt = content_to_text + "日本語に翻訳して200文字で要約してください。"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": request_to_gpt},
        ],
    )

    output_content = response.choices[0]["message"]["content"].strip()
    return output_content

output_content_text = run_gpt(article)
print(output_content_text)

イギリスのラシ・スナク首相は、人工知能（AI）のリスクについてはビッグテック企業に任せるべきではないと述べた。AI企業が自分たち自身の評価を行うべきではないと述べ、政府が行動を起こす必要があると述べた。AI安全サミットの開催前に、AIリスク管理に関するグローバル宣言が発表された。世界的な懸念が高まっているAIの高度な形態に関しては、まだ予測不可能な能力がある可能性があり、プライバシーの侵害、サイバー攻撃、雇用の追い出しといった潜在的なリスクがある。スナク首相は、AIは医療や学校などで大きな利益をもたらす可能性があると語ったが、「市民や国内のすべての人々の安全を確保するために必要なテストを行うためには、政府や外部の人々がその作業を行わなければならない」と述べた。


In [42]:
#ChatGPTでの翻訳機能
import openai

openai.api_key = OPENAI_API_KEY

def run_gpt(content_text_to_gpt):
    request_to_gpt = content_text_to_gpt + "日本語に翻訳して300文字で要約してください。"
    # APIの動作確認に使用
    # request_to_gpt = content_text_to_gpt + "日本語に翻訳して10文字で要約してください。"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": request_to_gpt},
        ],
    )

    output_content = response.choices[0]["message"]["content"].strip()
    return output_content

#output_content_text = run_gpt(article)
# APIの動作確認に使用
# output_content_text = run_gpt("今日は晴れでした。明日は晴れるかな")
#print(output_content_text)

In [43]:
print()

NameError: name 'output_content' is not defined